In [2]:
import fastbook
from fastbook import *

In [5]:
from torch.distributions.beta import Beta

class MyMixUp(Callback):
    "A handler class for implementing `MixUp` style scheduling"
    run_valid = False
    def __init__(self, alpha=0.5):
        self.distrib = Beta(tensor(alpha), tensor(alpha))

    def before_train(self):
        self.stack_y = getattr(self.learn.loss_func, 'y_int', False)
        print(f'self.stack_y: {self.stack_y}')
        if self.stack_y: self.old_lf,self.learn.loss_func = self.learn.loss_func,self.lf
        print(f'self.old_lf: {self.old_lf}, self.loss_func: {self.lf}')

    def after_train(self):
        if self.stack_y: self.learn.loss_func = self.old_lf

    def after_cancel_train(self):
        self.after_train()

    def before_batch(self):
        lam = self.distrib.sample((self.y.size(0),)).squeeze().to(self.x.device)
        print(f'self.distrib.sample(): {lam.shape}')
        lam = torch.stack([lam, 1-lam], 1)
        print(f'stack_lam: {lam.shape}')
        self.lam = lam.max(1)[0]
        print(f'self.lam: {lam.shape}')
        shuffle = torch.randperm(self.y.size(0)).to(self.x.device)
        xb1,self.yb1 = tuple(L(self.xb).itemgot(shuffle)),tuple(L(self.yb).itemgot(shuffle))
        print(f'xb1[0].shape: {xb1[0].shape}')
        print(f'self.yb1[0].shape: {self.yb1[0].shape}')
        nx_dims = len(self.x.size())
        print(f'nx_dims: {nx_dims}')
        self.learn.xb = tuple(L(xb1,self.xb).map_zip(torch.lerp,weight=unsqueeze(self.lam, n=nx_dims-1)))
        print(f'self.learn.xb[0].shape: {self.learn.xb[0].shape}')

        if not self.stack_y:
            ny_dims = len(self.y.size())
            print(f'not self stack: nx_dims: {ny_dims}')
            self.learn.yb = tuple(L(self.yb1,self.yb).map_zip(torch.lerp,weight=unsqueeze(self.lam, n=ny_dims-1)))
            print(f'not self stack: self.learn.yb[0].shape: {self.learn.yb[0].shape}, self.learn.yb[0].shape: {self.learn.yb[0].shape}')
        print(f'self.learn.yb[0]: {self.learn.yb[0]}')

    def lf(self, pred, *yb):
        if not self.training: return self.old_lf(pred, *yb)
        print(f'pred.shape: {pred.shape}, len(*yb): {len(*yb)}, yb[0].shape: {yb[0].shape}')
        print(f'self.lam: {self.lam}')
        with NoneReduce(self.old_lf) as lf:
            loss = torch.lerp(lf(pred,*self.yb1), lf(pred,*yb), self.lam)
        return reduce_loss(loss, getattr(self.old_lf, 'reduction', 'mean'))
    

In [6]:
from fastai.vision.all import *
#path = untar_data(URLs.PETS)/'images'
path = Path('/home/pawel/Pictures/oxford-iiit-pet/')

def is_cat(x): return x[0].isupper()
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat, item_tfms=Resize(28))

learn = cnn_learner(dls, resnet18, loss_func=CrossEntropyLossFlat(), metrics=error_rate, cbs=MyMixUp())
learn.fine_tune(1)

epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:01


self.stack_y: True
self.old_lf: FlattenedLoss of CrossEntropyLoss(), self.loss_func: <bound method MyMixUp.lf of MyMixUp>
self.distrib.sample(): torch.Size([64])
stack_lam: torch.Size([64, 2])
self.lam: torch.Size([64, 2])
xb1[0].shape: torch.Size([64, 3, 28, 28])
self.yb1[0].shape: torch.Size([64])
nx_dims: 4
self.learn.xb[0].shape: torch.Size([64, 3, 28, 28])
self.learn.yb[0]: TensorCategory([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred.shape: torch.Size([64, 1]), len(*yb): 64, yb[0].shape: torch.Size([64])
self.lam: tensor([0.9920, 0.7631, 0.8199, 0.7906, 0.7299, 0.9589, 0.6837, 0.9791, 0.7160, 0.9928, 0.6624, 0.9990, 0.9985, 0.9997, 0.8074, 0.9735, 0.9672, 0.9406, 0.8462, 0.5659, 0.9381, 0.7135, 0.8466, 0.9360,
        0.9871, 0.6665, 0.6361, 0.6362, 0.7857, 0.9867, 0.7272, 0.8160, 0.9995, 0.9917, 0.7767, 0.9858, 0.7439, 0.7536, 0.

epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:02


self.stack_y: True
self.old_lf: FlattenedLoss of CrossEntropyLoss(), self.loss_func: <bound method MyMixUp.lf of MyMixUp>
self.distrib.sample(): torch.Size([64])
stack_lam: torch.Size([64, 2])
self.lam: torch.Size([64, 2])
xb1[0].shape: torch.Size([64, 3, 28, 28])
self.yb1[0].shape: torch.Size([64])
nx_dims: 4
self.learn.xb[0].shape: torch.Size([64, 3, 28, 28])
self.learn.yb[0]: TensorCategory([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred.shape: torch.Size([64, 1]), len(*yb): 64, yb[0].shape: torch.Size([64])
self.lam: tensor([0.8526, 0.9077, 0.9991, 0.9083, 0.5336, 0.6136, 0.6970, 0.5269, 0.5187, 0.6651, 0.7535, 0.9840, 0.9260, 0.7264, 0.8301, 0.9078, 0.9173, 0.9974, 0.7876, 0.8584, 0.5664, 0.7352, 0.6590, 0.6829,
        0.8929, 0.9883, 0.8888, 0.8614, 0.7744, 0.7084, 0.8881, 0.9180, 0.5758, 0.5804, 0.5970, 0.8703, 0.9917, 0.7355, 0.

(#7393) [Path('/home/pawel/.fastai/data/oxford-iiit-pet/images/Bombay_160.jpg'),Path('/home/pawel/.fastai/data/oxford-iiit-pet/images/basset_hound_124.jpg'),Path('/home/pawel/.fastai/data/oxford-iiit-pet/images/wheaten_terrier_144.jpg'),Path('/home/pawel/.fastai/data/oxford-iiit-pet/images/basset_hound_95.jpg'),Path('/home/pawel/.fastai/data/oxford-iiit-pet/images/american_bulldog_66.jpg'),Path('/home/pawel/.fastai/data/oxford-iiit-pet/images/pug_27.jpg'),Path('/home/pawel/.fastai/data/oxford-iiit-pet/images/wheaten_terrier_95.jpg'),Path('/home/pawel/.fastai/data/oxford-iiit-pet/images/basset_hound_162.jpg'),Path('/home/pawel/.fastai/data/oxford-iiit-pet/images/Maine_Coon_104.jpg'),Path('/home/pawel/.fastai/data/oxford-iiit-pet/images/pomeranian_15.jpg')...]